In [1]:
import pandas as pd
import numpy as np
import datetime
from IPython import display

import matplotlib.pyplot as plt

In [2]:
test_df = pd.read_csv('datasets/test_df.csv').drop(columns='Unnamed: 0')
test_df['timestamp'] = test_df['timestamp'].apply(lambda x: datetime.datetime.fromisoformat(x))

train_df = pd.read_csv('datasets/train_df.csv').drop(columns='Unnamed: 0')
train_df['timestamp'] = train_df['timestamp'].apply(lambda x: datetime.datetime.fromisoformat(x))

## Proprocessing:
For leveraging predictive models the following features are created:
- $t_e$ - time since previous event
- $t_w$ - time since the beginning of week
- $t_t$ - time since the beginning of trace
- one hot encoded labels

In [3]:
import rl4pm_lib.preprocessing as preprocessing

column_feature = {'tt': 0, 'te': 1, 'tw': 2}
prepro = preprocessing.DfPreprocesser()
prepro.fit(train_df)
train_df_pr = prepro.transform(train_df)
test_df_pr = prepro.transform(test_df)

Also must scale continious (time related features)

In [4]:
scaler = preprocessing.PaperScaler(column_feature)
scaler.fit(train_df_pr)
train_df_pr_sc = scaler.transform(train_df_pr)
test_df_pr_sc = scaler.transform(test_df_pr)

Let's construct features as moving window

In [5]:
win_len = 2
max_window_len = 5

In [6]:
from rl4pm_lib.utils_supervised import make_window_features

In [7]:
test_df_pr_sc_win, test_labels, test_tes = make_window_features(test_df_pr_sc, win_len)
test_df_pr_sc_win.drop(columns=['trace_id'], inplace=True)

In [8]:
train_df_pr_sc_win, train_labels, train_tes = make_window_features(train_df_pr_sc, win_len)
train_df_pr_sc_win.drop(columns=['trace_id'], inplace=True)

Ok, features and targets are ready, let's do `.fit()`, `.predict())))))`

What methods will I use? Of course all, which were listed [here](https://github.com/lemikhovalex/DA_CV)

# Classification
## Random Forest Classifier

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [10]:
%%time
rfc_activ = RandomForestClassifier(n_estimators=130, max_depth=17, random_state=42, n_jobs=-1)

rfc_activ.fit(train_df_pr_sc_win, train_labels)
train_labels_hat = rfc_activ.predict(train_df_pr_sc_win)
test_labels_hat = rfc_activ.predict(test_df_pr_sc_win)

test_acc_rfc = accuracy_score(test_labels_hat, test_labels)
train_acc_rfc = accuracy_score(train_labels_hat, train_labels)
print(f'Random Forest Classifier\ntest  accuracy = {test_acc_rfc: .2f}\ntrain accuracy = {train_acc_rfc: .2f}')

Random Forest Classifier
test  accuracy =  0.77
train accuracy =  0.79
Wall time: 4.57 s


In [11]:
from sklearn.model_selection import GridSearchCV

In [16]:
%%time
param_grid = { 
    'n_estimators': np.linspace(180, 220, num=15, dtype=int),
    'max_depth' : np.linspace(10, 16, num=6, dtype=int),
}

model_gs = RandomForestClassifier(random_state=42)
CV_rfc = GridSearchCV(estimator=model_gs, param_grid=param_grid, scoring='accuracy', cv=7, n_jobs=-1)
CV_rfc.fit(train_df_pr_sc_win, train_labels)

Wall time: 30min 21s


GridSearchCV(cv=7, estimator=RandomForestClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': array([10, 11, 12, 13, 14, 16]),
                         'n_estimators': array([180, 182, 185, 188, 191, 194, 197, 200, 202, 205, 208, 211, 214,
       217, 220])},
             scoring='accuracy')

In [17]:
CV_rfc.best_params_

{'max_depth': 16, 'n_estimators': 182}

In [ ]:
%%time
rfc_activ_best = CV_rfc.best_estimator_
rfc_activ_best.fit(train_df_pr_sc_win, train_labels)

train_labels_hat = rfc_activ_best.predict(train_df_pr_sc_win)
test_labels_hat = rfc_activ_best.predict(test_df_pr_sc_win)

test_acc_rfc = accuracy_score(test_labels_hat, test_labels)
train_acc_rfc = accuracy_score(train_labels_hat, train_labels)

print(f'Random Forest Classifier, after grid search')
print(f'test  accuracy = {test_acc_rfc: .2f}\ntrain accuracy = {train_acc_rfc: .2f}')

Resuls some how are beter then in article on LSTM approach

## K Neighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate

In [ ]:
xgb_clf = KNeighborsClassifier()
grid_xgb = GridSearchCV(xgb_clf, {'n_neighbors': np.linspace(3, 30, 6, dtype=int),
                                  'weights': ['distance', 'uniform']
                                 },
                        n_jobs=-1, scoring='accuracy', cv=2, verbose=3)
grid_xgb.fit(train_df_pr_sc_win, train_labels)
print(grid_xgb.best_score_)
print(grid_xgb.best_params_)

Ok, next

## Grad Boost

In [ ]:
import xgboost as xgb

In [ ]:
param = {'max_depth': 17,  # the maximum depth of each tree
         'eta': 0.3,  # the training step for each iteration
         'objective': 'multi:softprob',  # error evaluation for multiclass training
         'num_class': 24,
         'n_jobs': -1,
         'n_estimators': 7  ,
         'tree_method': 'gpu_hist'       
        }

xgb_clf = xgb.XGBClassifier(**param)
xgb_clf.fit(train_df_pr_sc_win, train_labels)

train_labels_hat = xgb_clf.predict(train_df_pr_sc_win)
test_labels_hat = xgb_clf.predict(test_df_pr_sc_win)

test_acc_xgb = accuracy_score(test_labels_hat, test_labels)
train_acc_xgb = accuracy_score(train_labels_hat, train_labels)

print(f'XG boost Classifier')
print(f'test  accuracy = {test_acc_xgb: .2f}\ntrain accuracy = {train_acc_xgb: .2f}')

In [ ]:
param = {'objective': 'multi:softprob',  # error evaluation for multiclass training
         'num_class': 24,
         'tree_method': 'gpu_hist'
        }

xgb_clf = xgb.XGBClassifier(**param)
grid_xgb = GridSearchCV(xgb_clf, {'max_depth': np.linspace(3, 20, num=15, dtype=int),
                                  'n_estimators': np.linspace(3, 15, num=6, dtype=int),
                                  'eta': np.linspace(1e-2, 0.5, num=10, dtype=float)
                                 },
                        n_jobs=-1, scoring='accuracy', cv=2, verbose=3)
grid_xgb.fit(train_df_pr_sc_win, train_labels)
print(grid_xgb.best_score_)
print(grid_xgb.best_params_)

In [ ]:
param = {'objective': 'multi:softprob',  # error evaluation for multiclass training
         'num_class': 24,
         'tree_method': 'gpu_hist'
        }

xgb_clf = xgb.XGBClassifier(**param)
grid_xgb = GridSearchCV(xgb_clf, {'max_depth': np.linspace(5, 10, num=5, dtype=int),
                                  'n_estimators': np.linspace(3, 10, num=7, dtype=int),
                                  'eta': np.linspace(0.001, 0.3, num=20, dtype=float)
                                 },
                        n_jobs=-1, scoring='accuracy', cv=2, verbose=3)
grid_xgb.fit(train_df_pr_sc_win, train_labels)
print(grid_xgb.best_score_)
print(grid_xgb.best_params_)

In [ ]:
xgb_clf_best = grid_xgb.best_estimator_

In [ ]:
train_labels_hat = xgb_clf_best.predict(train_df_pr_sc_win)
test_labels_hat = xgb_clf_best.predict(test_df_pr_sc_win)

test_acc_xgb = accuracy_score(test_labels_hat, test_labels)
train_acc_xgb = accuracy_score(train_labels_hat, train_labels)

print(f'XG boost Classifier')
print(f'test  accuracy = {test_acc_xgb: .2f}\ntrain accuracy = {train_acc_xgb: .2f}')

## Classification results
Classique approaches povides results, which outperforms result, given in article as baseline, but...

- As been shown in visualization, there are a lot of loops, model that doesnt consern information on all the trail seems to be useless for end time prediction
- Accuracy is what been used for benchmarking in papers. Maybe it is not the best metric? Who knows, nevertheless this is a bench

# Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as MAE

In [ ]:
%%time
rfc_te = RandomForestRegressor(n_estimators=150, max_depth=17, random_state=42, n_jobs=-1)

rfc_te.fit(train_df_pr_sc_win, train_tes)
train_tes_hat = rfc_te.predict(train_df_pr_sc_win)
test_tes_hat = rfc_te.predict(test_df_pr_sc_win)

test_mae_rfr = MAE(test_tes_hat * scaler.scales['te'], test_tes * scaler.scales['te'])
train_mae_rfr = MAE(train_tes_hat * scaler.scales['te'], train_tes * scaler.scales['te'])
print(f'Random Forest Classifier\ntest  MAE = {test_mae_rfr / 60 / 60: .2f}h\ntrain MAE = {train_mae_rfr / 60 / 60: .2f}h')

Again better then papers)

In [ ]:
import xgboost as xgb
param = {'max_depth': 17,  # the maximum depth of each tree
         'eta': 0.3,  # the training step for each iteration
         'n_jobs': -1,
         'n_estimators': 7      
        }

xgb_reg = xgb.XGBRegressor(**param)
xgb_reg.fit(train_df_pr_sc_win, train_tes)

train_tes_hat = xgb_reg.predict(train_df_pr_sc_win)
test_tes_hat = xgb_reg.predict(test_df_pr_sc_win)

test_mae_xgb = MAE(test_tes_hat * scaler.scales['te'], test_tes * scaler.scales['te'])
train_mae_xgb = MAE(test_tes_hat * scaler.scales['te'], test_tes * scaler.scales['te'])

print(f'XG boost Regressor')
print(f'Random Forest Classifier\ntest  MAE = {test_mae_xgb / 3600 / 24: .2f}d\ntrain MAE = {train_mae_xgb / 3600 / 24: .2f}d')

XGB is not that cool

# No long activity!